In [1]:
from mxnet import image
from mxnet import gluon
from mxnet import nd
from mxnet.gluon import nn,Block
from mxnet import init
from mxnet.gluon.model_zoo import vision as models
import mxnet as mx
import numpy as np
import random
import sys
sys.path.append('..')
import utils
from time import time
start = time()
data_root = '../data'
image_root = data_root + '/CephalometricLandmark/RawImage'
# image_root = data_root + '/CephalometricLandmark/ContrastImage1'
txt_root = data_root + '/CephalometricLandmark/AnnotationsByMD'
model_params_root = data_root + '/CephalometricLandmark/model_params'

colormap = [[0,0,0],[128,0,0],[0,128,0], [128,128,0], [0,0,128],
            [128,0,128],[0,64,128],[128,128,128],[64,0,0],[192,0,0],
            [64,128,0],[192,128,0],[64,0,128],[192,0,128],
            [64,128,128],[192,128,128],[0,64,0],[128,64,0],
            [0,192,0],[128,192,0]]
rgb_mean = nd.array([0.485, 0.456, 0.406])
rgb_std = nd.array([0.229, 0.224, 0.225])

landmark_index = 3
expand_size = 320

In [2]:
def read_images(dataset_num=0):
    if dataset_num == 0:
        begin_index = 1
        end_index = 151
    elif dataset_num == 1:
        begin_index = 151
        end_index = 301
    else:
        begin_index = 301
        end_index = 401

    data = [None] * (end_index - begin_index)
    label = [None] * (end_index - begin_index)
    index = 0
    for i in range(begin_index, end_index):
        image_filename = image_root + "/%03d.bmp" % (i)
        data[index] = image.imread(image_filename)
        txt_filename = txt_root + '/400_junior' + "/%03d.txt" % i
        with open(txt_filename, 'r') as f:
            txts = f.read().split()
        x = int(txts[landmark_index].split(',')[0])
        y = int(txts[landmark_index].split(',')[1])
        label[index]= (x , y)
        index += 1
    return data,label

In [3]:
def crop_image_net0(images,labels):
    lenth = len(images)
    data = [None] * lenth
    label = [None] * lenth
    for i in range(len(images)):
        height = images[i].shape[0]
        width = images[i].shape[1]
        data[i] = images[i][height - 1920 : height , width - 1600:width]
        label[i] = (labels[i][0] - width + 1600,labels[i][1] - height + 1920)
    return data,label
        

In [4]:
def crop_image_net1(images,labels,centerx,centery):
    height = images.shape[0]
    width = images.shape[1]
    minx = centerx - 320
    miny = centery - 320
    if minx < 0:
        minx = 0
    if minx + 640 > width:
        minx = width - 640
    if miny < 0:
        miny = 0
    if miny + 640 > height:
        miny = height - 640
    data = images[miny : miny + 640 , minx : minx + 640]
    label = (labels[0] - minx,labels[1] - miny)
    return data,label

In [5]:
def crop_image_net2(images,labels,centerx,centery):
    height = images.shape[0]
    width = images.shape[1]
    minx = centerx - 32
    miny = centery - 32
    if minx < 0:
        minx = 0
    if minx + 64 > width:
        minx = width - 64
    if miny < 0:
        miny = 0
    if miny + 64 > height:
        miny = height - 64
    data = images[miny : miny + 64 , minx : minx + 64]
    label = (labels[0] - minx,labels[1] - miny)
    return data,label

In [6]:
test_images, test_labels = read_images(1)
test_images, test_labels = crop_image_net0(test_images, test_labels)
print(test_images[0].shape)

(1920, 1600, 3)


In [7]:
ctx = utils.try_all_gpus()

In [8]:
pretrained_net = models.resnet18_v2(pretrained=True)
net0 = nn.HybridSequential()
for layer in pretrained_net.features[:-2]:
    net0.add(layer)
with net0.name_scope():
    net0.add(
        nn.Conv2D(2, kernel_size=1),
        nn.Conv2DTranspose(2, kernel_size=64, padding=16,strides=32)
    )
net0[-1].initialize(init=init.Zero())
net0[-2].initialize(init=init.Xavier())
net0.collect_params().reset_ctx(ctx)
net0.load_params(filename = model_params_root + '/net0-mark%02d-resnet18-epochs100.json'%(landmark_index+1),ctx = ctx)

In [9]:
pretrained_net = models.resnet18_v2(pretrained=True)
net1 = nn.HybridSequential()
for layer in pretrained_net.features[:-2]:
    net1.add(layer)
with net1.name_scope():
    net1.add(
        nn.Conv2D(2, kernel_size=1),
        nn.Conv2DTranspose(2, kernel_size=64, padding=16,strides=32)
    )
net1[-1].initialize(init=init.Zero())
net1[-2].initialize(init=init.Xavier())
net1.collect_params().reset_ctx(ctx)
net1.load_params(filename = model_params_root + '/net1-mark%02d-resnet18-epochs100.json'%(landmark_index+1),ctx = ctx)

In [10]:
# pretrained_net = models.resnet18_v2(pretrained=True)
# net2 = nn.HybridSequential()
# for layer in pretrained_net.features[:-2]:
#     net2.add(layer)
# with net2.name_scope():
#     net2.add(
#         nn.Conv2D(2, kernel_size=1),
#         nn.Conv2DTranspose(2, kernel_size=64, padding=16,strides=32)
#     )
# net2[-1].initialize(init=init.Zero())
# net2[-2].initialize(init=init.Xavier())
# net2.collect_params().reset_ctx(ctx[0])
# net2.load_params(filename = model_params_root + '/net2-mark10-resnet18-epochs100.json',ctx = ctx[0])

In [11]:
def normalize_image(data):
    return (data.astype('float32') / 255 - rgb_mean) / rgb_std

In [12]:
def predict(im, mynet):
    data = normalize_image(im)
    data = data.transpose((2,0,1)).expand_dims(axis=0)
    yhat = mynet(data.as_in_context(ctx[0]))
    pred = nd.argmax(yhat, axis=1)
    return pred.reshape((pred.shape[1], pred.shape[2]))

def label2image(pred):
    x = pred.astype('int32').asnumpy()
    cm = nd.array(colormap).astype('uint8')
    return nd.array(cm[x,:])

In [13]:
def cal_point(result):
    if(len(np.where(result.asnumpy()>0)[1]) == 0):
        return -1,-1
    result_avgx = np.average(np.where(result.asnumpy()>0)[1])
    result_avgy = np.average(np.where(result.asnumpy()>0)[0])
    return int(round(result_avgx)), int(round(result_avgy))

In [14]:
def evaluate_acc(result, label):
    result_avgx, result_avgy = cal_point(result)
    if(result_avgx <=0 or result_avgy <= 0):
        return False,False,False,False
    
    d = pow((result_avgx - label[0]),2) + pow((result_avgy - label[1]),2)

    return (d < 400, d < 625, d < 900, d < 400)

In [15]:
n = len(test_images)
# n = 100
imgs = []
acc1 = 0
acc2 = 0
acc3 = 0
acc4 = 0

acc4_net0 = 0
acc4_net1 = 0
acc4_net2 = 0
# print(test_images[0])

for i in range(n):
#     print("================" + str(i) + "================")
    current_image = test_images[i]
    current_label = test_labels[i]
#     start = time()
    result0 = predict(current_image,net0)
    net0x,net0y = cal_point(result0)
#     print(net0x,net0y)
#     print(current_label)
    _, _ ,_ , f4 = evaluate_acc(result0,current_label)
    acc4_net0 += int(f4)
    current_image, current_label = crop_image_net1(current_image,current_label,net0x,net0y)
    
    
    result1 = predict(current_image,net1)
    net1x,net1y = cal_point(result1)
#     print(net1x,net1y)
#     print(current_label)
    _, _ ,_ ,f4 = evaluate_acc(result1,current_label)
    acc4_net1 += int(f4)
#     current_image, current_label = crop_image_net2(current_image,current_label,net1x,net1y)
    
#     result2 = predict(current_image,net2)
#     net2x,net2y = cal_point(result2)
#     print(net2x,net2y)
#     print(current_label)
#     _, _ ,_ , f4 = evaluate_acc(result2,current_label)
#     acc4_net2 += int(f4)
    


#     print(type(np.where(result.asnumpy()>0)[0]))
#     print(np.where(test_labels[i].asnumpy()>0))
#     pred = label2image(predict(x))
#     label = nd.zeros((test_images[i].shape[0], test_images[i].shape[1]))
#     minx = test_labels[i][0] - expand_size
#     maxx = test_labels[i][0] + expand_size
#     if minx < 0:
#         minx = 0
#     if maxx >= 1600:
#         maxx = 1600-1

#     miny = test_labels[i][1] - expand_size
#     maxy = test_labels[i][1] + expand_size
#     if miny < 0:
#         miny = 0
#     if maxy >= 1920:
#         maxy = 1920-1
#     label[miny:maxy, minx:maxx] = 1
#     imgs += [x, pred, label2image(label)]
print(acc4_net0/n,"  ", acc4_net1/n,"  ", acc4_net2/n)
# utils.show_images(imgs, nrows=n, ncols=3, figsize=(6,20))
print('耗时',str(time() - start),'秒')

0.06666666666666667    0.68    0.0
耗时 50.48726963996887 秒
